In [1]:
from SBM_attributed import *
import torch
import networkx as nx
from divergences import * 
from sklearn.metrics import rand_score, calinski_harabasz_score, mutual_info_score, accuracy_score

In [2]:
### Generate benchmark
P = np.array([[0.8, 0.2, 0.3],[0.2, 0.7, 0.4],[0.3, 0.4, 0.6]])
c = 3
n = 100
N = c*n
delta = 10
d = 2
dim = c*d
_,A,X = generate_benchmark(P,c,delta=delta,n=n,dim=d,sample_along_direction=True)
A = torch.tensor(A)
X = torch.tensor(X)

In [3]:
### initialize variables
B = torch.eye(c,dtype=torch.float, requires_grad=True)
W = torch.zeros((N,c),dtype=torch.float, requires_grad=True)
indexes = torch.randint(low=0,high=c,size=(N,))
with torch.no_grad():
    for index,col in enumerate(indexes):
        W[index].index_fill_(0, col, 1)
mu = torch.tensor(np.random.normal(size=(c,dim)), requires_grad=True)

In [4]:
phi_net = get_phi("euclidean",elementwise=True)[0]
phi_data = get_phi("euclidean")

In [5]:
def obj_func(W,B,mu,norm):
    net_divergence = torch.sum(phi_net(A,W@B@W.T), axis=1)
    data_divergence = torch.sum(torch.multiply(W, pairwise_bregman(X, mu, phi_data)),axis=1)
    K = torch.stack((net_divergence,data_divergence),axis=-1)
    return torch.linalg.matrix_norm(K,ord=-1,keepdim=False)

In [6]:
norm = -1
classes_old = classes = None
convergence_cnt = 0
convergence_threshold = 64
iter = 0
while True:
    iter += 1
    loss = obj_func(W,B,mu,norm)
    loss.backward()
    with torch.no_grad():
        W -= W.grad 
        B -= B.grad 
        mu -= mu.grad 
        # Set the gradients to zero
        W.grad.zero_()
        B.grad.zero_()
        mu.grad.zero_()
#         W -= W.min()
#         W /= W.max()
#         B -= B.min()
#         B /= B.max()
        #anneal s value
        if iter % 2 == 0:
            if norm > -1.0:
                norm += -.2
            elif norm > -120.0:
                norm *= 1.06
        if classes is not None:
            classes_old = classes
        classes = torch.argmax(W, axis=1)
        if classes_old is not None and classes is not None and torch.equal(classes_old, classes):
            convergence_cnt += 1
        else:
            convergence_cnt = 0
        if convergence_cnt == convergence_threshold:
            print("point assignments have converged")
            break
        #norm *= 1.001

point assignments have converged


In [7]:
pred_labels = torch.argmax(W, axis=1).detach().numpy()
true_labels = [0]*n + [1]*n + [2]*n
print(rand_score(true_labels, pred_labels),mutual_info_score(true_labels, pred_labels))

0.8683612040133779 0.8366210750785297
